In [7]:
!pip install deap

In [8]:
#import required packages

import operator
import math
import random

import numpy

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
import threading

In [9]:
# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return 1

# Define primitives and set
pset = gp.PrimitiveSet("MAIN", 7)
pset.addPrimitive(operator.add, 2)
pset.addPrimitive(operator.sub, 2)
pset.addPrimitive(operator.mul, 2)
pset.addPrimitive(protectedDiv, 2)
pset.addPrimitive(operator.neg, 1)
pset.addEphemeralConstant("rand101", partial(random.randint, -1, 1))
pset.renameArguments(ARG0='x', ARG1='p', ARG2='z', ARG3='f', ARG4='h', ARG5='k',ARG6='d')

# Define fitness and individual
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

# Create toolbox
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=2)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

# Define evaluation function
def evalSymbReg(individual, data):
    func = toolbox.compile(expr=individual)

    # Assume data is a list of rows, where each row is a list of values
    total_hits=0;
    for row in data:
        # Extract features (excluding the target variable)
        features = row[:-1]

        # Evaluate the expression
        predicted = func(*features)

        # Convert the predicted value to a category (you may need to adjust these thresholds)
        predicted_category = 1.0 if predicted >= 0.0 else 0.0  #first and last float value is the rice

        # Extract the actual category from the last column
        actual_category = row[-1]

        # Calculate error based on incorrect categorization
        hit = 1 if predicted_category == actual_category else 0

        # Add the error to the list
        total_hits+=hit;
    # Return the average hits
    return (1-(total_hits / len(data)))**2,


def evalHit(individualError, data):
    func = toolbox.compile(expr=individualError)

    total_hits = 0
    for row in data:
        features = row[:-1]
        predicted = func(*features)
        predicted_category = 1 if predicted >= 0 else 0
        actual_category = row[-1]
        hit = 1 if predicted_category == actual_category else 0
        total_hits += hit

    return total_hits  # Return the total number of hits as an integer

def evalPrediction(individual, data):
    func = toolbox.compile(expr=individual)

    predictions = []

    for i, row in enumerate(data):
        features = row[:-1]
        predicted = func(*features)
        predicted_category = 1 if predicted >= 0 else 0 #simply copying the data from the testing data set
        actual_category = row[-1]

        Evaluation="true" if predicted_category==actual_category else "false"
        predictions.append((i, actual_category, predicted_category, Evaluation))

    return predictions

# Function to convert string data to float array
def string_to_float_array(data_list):
    float_array = []
    for row in data_list:
        values = row.split(',')
        if values[-1].strip() == 'Cammeo':
            values[-1] = 1.0
        elif values[-1].strip() == 'Osmancik':
            values[-1] = 0.0
        else:
            values[-1] = float('nan')
        float_array.append(list(map(float, values)))
    return float_array

# Load data from file
with open("Rice_data.txt", 'r') as file:
    float_values = file.readlines()
    random.shuffle(float_values)
    split_index = int(len(float_values) * 0.3)###############################################################################################################
    training_set = float_values[:split_index]
    testing_set = float_values[split_index:]
    float_array = string_to_float_array(training_set)

# Register evaluation function
toolbox.register("evaluate", evalSymbReg, data=float_array)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=2)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)
toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))

In [10]:
# Define thread function for evaluating individuals
def evaluate_individuals(individuals, data, results):
    for ind in individuals:
        result = evalSymbReg(ind, data)
        results.append(result)

# Define main function
def main(seed):
    random.seed(seed)
    pop = toolbox.population(n=800) #should be 800
    hof = tools.HallOfFame(1)
    stats_fit = tools.Statistics(lambda ind: evalSymbReg(ind, float_array))
    stats_size = tools.Statistics(len)
    mstats = tools.MultiStatistics(fitness=stats_fit, size=stats_size)
    mstats.register("avg", numpy.mean)
    mstats.register("std", numpy.std)
    mstats.register("min", numpy.min)
    mstats.register("max", numpy.max)

    # Number of threads to use
    num_threads = 16  # Adjust as needed

    # Split the population into chunks for parallel evaluation
    chunk_size = len(pop) // num_threads
    pop_chunks = [pop[i:i+chunk_size] for i in range(0, len(pop), chunk_size)]

    # Create lists to store evaluation results
    results = [[] for _ in range(num_threads)]

    # Create and start threads
    threads = []
    for i in range(num_threads):
        thread = threading.Thread(target=evaluate_individuals, args=(pop_chunks[i], float_array, results[i]))
        threads.append(thread)
        thread.start()

    # Wait for all threads to finish
    for thread in threads:
        thread.join()

    # Combine results from all threads
    combined_results = [result for sublist in results for result in sublist]

    # Assign fitness values to individuals
    for ind, result in zip(pop, combined_results):
        ind.fitness.values = result

    # Perform evolution #should be 0.70,0.3,50
    pop, log = algorithms.eaSimple(pop, toolbox, 0.70, 0.30, 50, stats=mstats, halloffame=hof, verbose=True)############################################################################

    # Retrieve fittest individual
    fittest_individual = hof[0]
    fittest_fitness = fittest_individual.fitness.values[0]
    print("Fittest individual:", fittest_individual)
    print("Fitness:", fittest_fitness)


    # Convert testing set to float array
    testing_float_array = string_to_float_array(testing_set)
    # Evaluate fittest individual on testing dataset

    train_hits = evalHit(fittest_individual, float_array)
    train_accuracy = (train_hits / len(float_array))

    test_hits = evalHit(fittest_individual, testing_float_array)
    test_accuracy = (test_hits / len(testing_float_array))

    print(f"Training Accuracy: {train_hits} / {len(float_array)} = {train_accuracy}")
    print(f"Testing Accuracy: {test_hits} / {len(testing_float_array)} = {test_accuracy}")
    print(f"Fitness of Best individual on Testing set: {(1-(test_hits / len(testing_float_array)))**2}")


    # Evaluate fittest individual on testing dataset
    predictions = evalPrediction(fittest_individual, testing_float_array)

    CameoCounter=0;
    OsmancikCounter=0;
    WrongCameoCounter=0;
    WrongOsmancikCounter=0;
    # Print predictions
    print("Predictions:")
    for prediction in predictions:
      # print("Index:", prediction[0], "Actual category:", prediction[1], "Predicted category:", prediction[2], "Evaluation: ",prediction[3])
      if prediction[1]==1 and prediction[3]=="true": CameoCounter=CameoCounter+1;
      if prediction[1]==0 and prediction[3]=="true": OsmancikCounter=OsmancikCounter+1;
      if prediction[1]==1 and prediction[3]=="false": WrongCameoCounter=WrongCameoCounter+1;
      if prediction[1]==0 and prediction[3]=="false": WrongOsmancikCounter=WrongOsmancikCounter+1;
    print("how many Cammeo it got right: ", CameoCounter, " how many Osmancik it got right: ",OsmancikCounter)
    print("how many Cammeo it got wrong: ", WrongCameoCounter, " how many Osmancik it got wrong: ",WrongOsmancikCounter)
    return pop, log, hof

# Number of runs
num_runs = 5
for run in range(1, num_runs + 1):
    seed_value = run
    pop, log, hof = main(seed_value)




   	      	                                 fitness                                 	                      size                     
   	      	-------------------------------------------------------------------------	-----------------------------------------------
gen	nevals	avg     	gen	max     	min     	nevals	std      	avg  	gen	max	min	nevals	std    
0  	0     	0.270318	0  	0.316468	0.178567	0     	0.0603836	3.955	0  	7  	2  	0     	1.64559
1  	612   	0.24104 	1  	0.316468	0.151575	612   	0.0612826	4.20125	1  	13 	1  	612   	2.09899
2  	639   	0.233817	2  	0.316468	0.191358	639   	0.0591167	4.7375 	2  	17 	1  	639   	2.57363
3  	635   	0.22658 	3  	0.357066	0.178567	635   	0.0563615	5.21125	3  	20 	1  	635   	2.90932
4  	639   	0.221455	4  	0.316468	0.153626	639   	0.0535442	5.805  	4  	21 	1  	639   	3.44231
5  	620   	0.221605	5  	0.369724	0.153626	620   	0.053775 	6.145  	5  	27 	1  	620   	3.70628
6  	617   	0.223309	6  	0.377209	0.153626	617   	0.054935 	6.49625	6  	27 	1  	6

KeyboardInterrupt: 